In [7]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()
from langchain_core.runnables import RunnableLambda, RunnableConfig
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.messages import BaseMessage
from langchain_community.tools.tavily_search import TavilySearchResults


In [8]:
GOOGLE_API_KEY = os.getenv("GOOGLE_GEMINI_API")

In [9]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [10]:
graph_builder = StateGraph(State)

In [13]:

tool = TavilySearchResults(max_results=2)
tools = [tool]



In [14]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",  # or "gemini-pro" for the older version
    temperature=0.3,
    google_api_key=GOOGLE_API_KEY 
)


In [ ]:
### Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)

In [16]:
def chatbot(state : State):
     return {"messages": [llm.invoke(state["messages"])]}

In [17]:
graph_builder.add_node("chatbot",chatbot)

Next we need to create a function to actually run the tools if they are called. We'll do this by adding the tools to a new node.

Below, we implement a BasicToolNode that checks the most recent message in the state and calls tools if the message contains tool_calls. It relies on the LLM's tool_calling support, which is available in Anthropic, OpenAI, Google Gemini, and a number of other LLM providers.

We will later replace this with LangGraph's prebuilt ToolNode to speed things up, but building it ourselves first is instructive.

In [20]:
import json
from langchain_core.messages import ToolMessage

In [ ]:
class BasicToolNode:
     """A node that runs the tools requested in the last AIMessage."""

     def __init__(self,tools:list) ->None:
          self.tools_by_name = {tool.name:tool for tool in tools}

     def __call__(self,inputs:dict):
          if messages := inputs.get("messages", []):
            message = messages[-1]
          else:
            raise ValueError("No message found in input")  
          
          outputs=[]

          for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
          
        
               



IndentationError: expected an indented block after 'for' statement on line 15 (132960269.py, line 19)